# Documents DB File

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"
file_name = "../data/raw/fintech-scopus.csv"
import_scopus_file_to_directory(file_name, directory)

# !cat ../data/processed/debug/filter.yaml

# Data Cleaning

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# clean_institution_fields(dirpath="../data/processed/debug")
# clean_keywords_fields(dirpath="../data/processed/debug")

# apply_thesaurus(dirpath_or_records="../data/processed/debug", thesaurus_filepath="../data/processed/debug/keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus")
# create_thesaurus(dirpath_or_records="../data/processed/debug", column="keywords", thesaurus_filepath="../data/processed/debug/test_thesaurus.txt")

# Basic Analysis

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"


# core_authors_report(directory) 
core_sources(directory)
# coverage_report(directory)
# summary(directory)
# most_cited_documents(directory)

# bradford_plot(dirpath_or_records="../data/processed/debug")
# lotka_plot(dirpath_or_records="../data/processed/debug")


# terms_table(dirpath_or_records="../data/processed/debug", column='authors')
# worldmap(dirpath_or_records="../data/processed/debug", metric='num_records')
# time_table(dirpath_or_records="../data/processed/debug")

# terms_per_year_table(dirpath_or_records="../data/processed/debug", column='countries')

# TermAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', top_by='num_records').table_
# TimeAnalyzer(dirpath_or_records="../data/processed/debug").barh(column='num_documents', cmap="Blues")
# worldmap(count_records_by_term(dirpath_or_records="../data/processed/debug", column="countries"))


# BigraphAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', by='countries').table_

In [ ]:





# apply_thesaurus(directory_or_records="../data/processed/debug", thesaurus_filepath="../data/processed/debug/keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus")
# association_index(co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords"), association='jaccard')

# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='author_keywords')
# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='countries')



# create_thesaurus(directory_or_records="../data/processed/debug", column="keywords", thesaurus_filepath="../data/processed/debug/test_thesaurus.txt")
# impact_analysis(directory_or_records="../data/processed/debug", column="authors")
# tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords")
# tfidf_matrix(tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords"))

# collaboration_analysis(directory_or_records="../data/processed/debug", column="authors")

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/documents.csv").iso_source_name.value_counts()

In [ ]:
pd.read_csv("../data/processed/debug/documents.csv").columns